# **Importing the needed libraries**

In [ ]:
!pip install fasttext

In [ ]:
!pip install gensim

In [ ]:
!pip install --upgrade numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 104.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.13.1 requires numpy<2.3,>=1.22.4, but you have numpy 2.3.0 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.3.0 which is incompatible.
cupy-cuda12x 13.3.0 requires numpy<2.3,>=1.22, but you have numpy 2.3.0 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.3.0 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.3.0 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version 

In [ ]:
!pip uninstall numpy -y
!pip uninstall gensim -y
!pip install numpy==1.25.2
!pip install gensim

Found existing installation: numpy 2.3.0
Uninstalling numpy-2.3.0:
  Successfully uninstalled numpy-2.3.0
Found existing installation: gensim 4.3.3
Uninstalling gensim-4.3.3:
  Successfully uninstalled gensim-4.3.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 123.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.25.2 which is incompatible.
blosc2 3.3.4 requires numpy>=1.26, but you have numpy 1.25.2 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.25.2 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.


  Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.7 MB)
^C


In [ ]:
!pip install camel-tools

In [ ]:
import re
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from nltk.stem.isri import ISRIStemmer
import gensim.downloader as api
import fasttext
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.utils.dediac import dediac_ar
from camel_tools.morphology.database import MorphologyDB
from camel_tools.morphology.analyzer import Analyzer
from sklearn.naive_bayes import MultinomialNB
from gensim.models import FastText


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

# **Reading the data**

In [ ]:
df1 = pd.read_csv('altibbi_specialty_data.csv')
df1.head(5)

,specialty_id,name_ar,question_body
0,23,طب عيون,استشاره عيون
1,14,جراحة العظام والمفاصل,السلام عليكم ممكن دكتور مفاصل واعصاب
2,14,جراحة العظام والمفاصل,عندي نقص فيتامين د هل ممكن استخدم معه كالسيوم
3,23,طب عيون,عمليه الحول للكبار
4,14,جراحة العظام والمفاصل,ألم بالكتف الايسر من فترة


# **Preprocessing the data**

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92559 entries, 0 to 92558
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   specialty_id   92559 non-null  int64 
 1   name_ar        92559 non-null  object
 2   question_body  92559 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.1+ MB


In [ ]:
df1.duplicated()
#There are no duplicated rows

,0
0,False
1,False
2,False
3,False
4,False
...,...
92554,False
92555,False
92556,False
92557,False


In [ ]:
#I will be using only 30,000 rows since this is a large dataset
from sklearn.model_selection import train_test_split
df, _ = train_test_split(df1, stratify=df1['name_ar'], test_size=0.675, random_state=42)
#This means to capture the same label disribution as the full dataset

#the shape and class balance
print(df.shape)
print(df['name_ar'].value_counts(normalize=True))

(30081, 3)
name_ar
جراحة العظام والمفاصل    0.275224
الطب النفسي              0.201223
طب اسنان                 0.201223
طب عيون                  0.173332
تغذية                    0.148998
Name: proportion, dtype: float64


In [ ]:
!camel_data -i all
#getting all camel databases and sets

No new packages will be installed.


In [ ]:
arabic_stopwords = set(stopwords.words('arabic'))
stemmer = ISRIStemmer()
db = MorphologyDB.builtin_db()
analyzer = Analyzer(db)

def preprocess_text(text):
    # REGEX: normalizing arabic letters
    text = re.sub(r'[إأآ]', 'ا', text)
    text = re.sub(r'ؤ', 'و', text)
    text = re.sub(r'ئ', 'ي', text)
    text = re.sub(r'ء', '', text)
    text = re.sub(r'ة', 'ه', text)

    # CAMEL remove diacritics
    text = dediac_ar(text)

    # REGEX remove punctuation
    text = re.sub(rf"[{re.escape(string.punctuation)}]", '', text)

    # CAMeL: Tokenize
    tokens = simple_word_tokenize(text)

    #Removing tokens that are just numbers
    tokens = [word for word in tokens if not word.isdigit()]

    # Removing stopwords
    tokens = [word for word in tokens if word not in arabic_stopwords]

    # lemmatizaton using camel analyzer if possible
    processed = []
    for word in tokens:
      analysis = analyzer.analyze(word)
      if analysis:
        lemma = analysis[0].get('lemma') or analysis[0].get('stem') or stemmer.stem(word)
        lemma = dediac_ar(lemma)  # remove diacritics from analyzer output
        processed.append(lemma)
      else:
        processed.append(stemmer.stem(word))

    return ' '.join(processed)

In [ ]:
#applying the preprocessing
df['Clean'] = df['question_body'].astype(str).apply(preprocess_text)

In [ ]:
df.head()

,specialty_id,name_ar,question_body,Clean
86387,91,الطب النفسي,حالتي أنني في منزلي ولاكن لااستطيع القيام بنشا...,حال آن منزل ولا لااستطيع قيام نشاط يومي طمأنين...
78464,18,طب اسنان,عندي الأسنان,عند آسن
61249,23,طب عيون,عيون 0000,عيون
16863,14,جراحة العظام والمفاصل,عندي وجع في ظهري مااعرف مكانه تحديدًا تقريبًا ...,عند وجع ظهر مااعرف مكان تحديد تقريب أعلى وسط ر...
53245,23,طب عيون,ارى خط ابيض عند النظر للاضاءة البيضاء ويختفي,آر خط أبيض نظر اضه بيض ختفي


In [112]:
all_raw_tokens = []
for text in df1['question_body']:
    all_raw_tokens.extend(simple_word_tokenize(text))
unique_raw_tokens = set(all_raw_tokens)
vocab_size_before = len(unique_raw_tokens)

print(f"Vocabulary size BEFORE preprocessing: {vocab_size_before}")

#Vocabulary size after preprocessing
df['question_body'] = df1['question_body'].apply(preprocess_text)

all_processed_tokens = []
for text in df['question_body']:
    all_processed_tokens.extend(text.split())
unique_processed_tokens = set(all_processed_tokens)
vocab_size_after = len(unique_processed_tokens)

print(f"Vocabulary size AFTER preprocessing: {vocab_size_after}")

reduction_percentage = ((vocab_size_before - vocab_size_after) / vocab_size_before) * 100
print(f"Vocabulary reduction: {reduction_percentage:.2f}%")

Vocabulary size BEFORE preprocessing: 76129
Vocabulary size AFTER preprocessing: 14004
Vocabulary reduction: 81.60%


# **Traditional Embeddings**

## **TF-IDF**

In [113]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['Clean'])
tf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
tf_df

,1000mg,100mg,100كلجم,100كيلو,100وابيها,105وابغا,105وطولي,107عايز,109كيلو,10mg,...,ٱهتمام,چل,ڤيتالايف,ڤيتام,ڤيروس,کمداتـ,گسرنی,گيب,ﯙبرگـاتہ,ﺑﮧـس
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30076,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30077,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30078,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30079,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### **Applying SVM to TF-IDF(A baseline model)**

In [114]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['Clean'], df['name_ar'], test_size=0.2, random_state=42, stratify=df['name_ar'])

In [115]:
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [116]:
from sklearn.svm import LinearSVC
svm = LinearSVC()
svm.fit(X_train_tfidf, y_train)

LinearSVC()

In [117]:
y_pred = svm.predict(X_test_tfidf)

In [118]:
from sklearn.metrics import classification_report
report_dict = classification_report(y_test, y_pred, output_dict=True)
report_df = pd.DataFrame(report_dict).transpose()
report_df

,precision,recall,f1-score,support
الطب النفسي,0.787402,0.908340,0.843558,1211.000000
تغذية,0.921117,0.847098,0.882558,896.000000
جراحة العظام والمفاصل,0.903614,0.905797,0.904704,1656.000000
طب اسنان,0.946413,0.904211,0.924831,1211.000000
طب عيون,0.947906,0.889741,0.917903,1043.000000
accuracy,0.894466,0.894466,0.894466,0.894466
macro avg,0.901290,0.891038,0.894711,6017.000000
weighted avg,0.899123,0.894466,0.895439,6017.000000


## ***One-Hot Encoding***

In [119]:
tokenized_sentences = [sentence.split() for sentence in df['Clean']]

from collections import Counter
# Count all word frequencies
word_counts = Counter(word for sentence in tokenized_sentences for word in sentence)
most_common_words = sorted(word_counts.keys())

# Build vocabulary mapping
word_to_index = {word: idx for idx, word in enumerate(most_common_words)}
vocab_size = len(word_to_index)

# Filter and index only known words
indexed_sentences = [
    [word_to_index[word] for word in sentence if word in word_to_index]
    for sentence in tokenized_sentences
]


In [120]:
hot_vectors = np.zeros((len(indexed_sentences), vocab_size))

for i, sentence in enumerate(indexed_sentences):
    for idx in sentence:
        hot_vectors[i][idx] = 1

In [121]:
hot_vectors.shape

(30081, 14004)

In [122]:
X_train, X_test, y_train, y_test = train_test_split(
    hot_vectors, df['name_ar'], test_size=0.2, random_state=42, stratify=df['name_ar']
)
# Training SVM
svm_onehot = LinearSVC()
svm_onehot.fit(X_train, y_train)

# evaluate
y_pred = svm_onehot.predict(X_test)
report = classification_report(y_test, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()

In [123]:
report_df

,precision,recall,f1-score,support
الطب النفسي,0.778917,0.878613,0.825766,1211.000000
تغذية,0.890071,0.840402,0.864524,896.000000
جراحة العظام والمفاصل,0.896970,0.893720,0.895342,1656.000000
طب اسنان,0.935567,0.899257,0.917053,1211.000000
طب عيون,0.931382,0.884947,0.907571,1043.000000
accuracy,0.882333,0.882333,0.882333,0.882333
macro avg,0.886581,0.879388,0.882051,6017.000000
weighted avg,0.885916,0.882333,0.883239,6017.000000


## **TF-IDF with Traditional Classification Models**

### **Naive Bayes**

In [124]:
nb = MultinomialNB()
nb.fit(X_train_tfidf, y_train)

y_pred_nb = nb.predict(X_test_tfidf)

report_nb = classification_report(y_test, y_pred_nb, output_dict=True)
pd.DataFrame(report_nb).transpose().round(3)

,precision,recall,f1-score,support
الطب النفسي,0.823,0.866,0.844,1211.00
تغذية,0.956,0.808,0.876,896.00
جراحة العظام والمفاصل,0.817,0.935,0.872,1656.00
طب اسنان,0.940,0.900,0.919,1211.00
طب عيون,0.951,0.849,0.897,1043.00
accuracy,0.880,0.880,0.880,0.88
macro avg,0.897,0.872,0.882,6017.00
weighted avg,0.887,0.880,0.881,6017.00


### **Logisitc Regression**

In [125]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_tfidf, y_train)

y_pred_lr = lr.predict(X_test_tfidf)
report_dict = classification_report(y_test, y_pred, output_dict=True)
pd.DataFrame(report_dict).transpose()

,precision,recall,f1-score,support
الطب النفسي,0.778917,0.878613,0.825766,1211.000000
تغذية,0.890071,0.840402,0.864524,896.000000
جراحة العظام والمفاصل,0.896970,0.893720,0.895342,1656.000000
طب اسنان,0.935567,0.899257,0.917053,1211.000000
طب عيون,0.931382,0.884947,0.907571,1043.000000
accuracy,0.882333,0.882333,0.882333,0.882333
macro avg,0.886581,0.879388,0.882051,6017.000000
weighted avg,0.885916,0.882333,0.883239,6017.000000


# **Neural Embeddings**

## ***Word2Vec CBow***

In [127]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ar.300.vec.gz

--2025-06-10 17:45:25--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ar.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.169.121.81, 3.169.121.57, 3.169.121.110, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.169.121.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1272365870 (1.2G) [binary/octet-stream]
Saving to: ‘cc.ar.300.vec.gz’

cc.ar.300.vec.gz    100%[===================>]   1.18G  33.1MB/s    in 33s     

2025-06-10 17:45:59 (36.7 MB/s) - ‘cc.ar.300.vec.gz’ saved [1272365870/1272365870]



In [128]:
!gunzip cc.ar.300.vec.gz

In [129]:
tokenized_sentences = [sentence.split() for sentence in df['Clean']]

In [130]:
from gensim.models import KeyedVectors
w2v_model = KeyedVectors.load_word2vec_format("cc.ar.300.vec", binary=False)

In [131]:
def sentence_vector(sentence, model):
    words = sentence.split()
    valid_words = [word for word in words if word in model]
    if not valid_words:
        return np.zeros(model.vector_size)
    return np.mean([model[word] for word in valid_words], axis=0)

Cbow_vectors = np.array([sentence_vector(text, w2v_model) for text in df['Clean']])

In [132]:
X_train, X_test, y_train, y_test = train_test_split(
    Cbow_vectors, df['name_ar'],
    test_size=0.2, stratify=df['name_ar'], random_state=42
)
svm_pretrained_w2v = LinearSVC()
svm_pretrained_w2v.fit(X_train, y_train)
y_pred = svm_pretrained_w2v.predict(X_test)

report = classification_report(y_test, y_pred, output_dict=True)
pd.DataFrame(report).transpose().round(3)

,precision,recall,f1-score,support
الطب النفسي,0.752,0.843,0.795,1211.000
تغذية,0.900,0.800,0.847,896.000
جراحة العظام والمفاصل,0.841,0.882,0.861,1656.000
طب اسنان,0.896,0.851,0.873,1211.000
طب عيون,0.905,0.847,0.875,1043.000
accuracy,0.849,0.849,0.849,0.849
macro avg,0.859,0.844,0.850,6017.000
weighted avg,0.854,0.849,0.850,6017.000


# ***FastText***

In [133]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ar.300.bin.gz

--2025-06-10 17:53:57--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ar.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.169.121.110, 3.169.121.57, 3.169.121.107, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.169.121.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4500982519 (4.2G) [application/octet-stream]
Saving to: ‘cc.ar.300.bin.gz’

cc.ar.300.bin.gz    100%[===================>]   4.19G   134MB/s    in 35s     

2025-06-10 17:54:32 (122 MB/s) - ‘cc.ar.300.bin.gz’ saved [4500982519/4500982519]



In [134]:
!gunzip cc.ar.300.bin.gz

In [135]:
import fasttext
model = fasttext.load_model('cc.ar.300.bin')

In [136]:
def sentence_vector(sentence, model):
    words = sentence.split()
    valid_words = [word for word in words if word in model]
    if not valid_words:
        return np.zeros(model.get_dimension())
    return np.mean([model[word] for word in valid_words], axis=0)

In [137]:
fasttext_vectors = np.array([sentence_vector(sent, model) for sent in df['Clean']])

In [138]:
X_train, X_test, y_train, y_test = train_test_split(
    fasttext_vectors, df['name_ar'],
    test_size=0.2, stratify=df['name_ar'], random_state=42
)

svm_ft = LinearSVC()
svm_ft.fit(X_train, y_train)

y_pred = svm_ft.predict(X_test)
report = classification_report(y_test, y_pred, output_dict=True)
pd.DataFrame(report).transpose().round(3)

,precision,recall,f1-score,support
الطب النفسي,0.752,0.843,0.795,1211.000
تغذية,0.900,0.800,0.847,896.000
جراحة العظام والمفاصل,0.841,0.882,0.861,1656.000
طب اسنان,0.895,0.851,0.872,1211.000
طب عيون,0.905,0.847,0.875,1043.000
accuracy,0.849,0.849,0.849,0.849
macro avg,0.859,0.844,0.850,6017.000
weighted avg,0.854,0.849,0.850,6017.000


## **FastText with Neural Language Models**

# ***LSTM***

In [139]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenized_sentences = [sentence.split() for sentence in df['Clean']]
max_len = 50

In [140]:
embed_dim = model.get_dimension()

def sentence_to_vector_sequence(sentence, model, max_len):
    vectors = []
    for word in sentence[:max_len]:
        if word in model:
            vectors.append(model[word])
        else:
            vectors.append(np.zeros(embed_dim))

    # Pad if too short
    while len(vectors) < max_len:
        vectors.append(np.zeros(embed_dim))

    return vectors

X_lstm = np.array([sentence_to_vector_sequence(sentence, model, max_len)
                   for sentence in tokenized_sentences])

In [141]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

le = LabelEncoder()
y_encoded = le.fit_transform(df['name_ar'])
y_categorical = to_categorical(y_encoded)

In [142]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_lstm, y_categorical, test_size=0.2, stratify=y_encoded, random_state=42
)

In [143]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model_lstm = Sequential()
model_lstm.add(LSTM(128, input_shape=(max_len, embed_dim)))
model_lstm.add(Dropout(0.3))
model_lstm.add(Dense(y_categorical.shape[1], activation='softmax'))

model_lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_lstm.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 128)            │       219,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 220,293 (860.52 KB)

 Trainable params: 220,293 (860.52 KB)

 Non-trainable params: 0 (0.00 B)

In [144]:
history = model_lstm.fit(X_train, y_train, validation_split=0.1, epochs=10, batch_size=64)

Epoch 1/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 37s 102ms/step - accuracy: 0.3954 - loss: 1.3431 - val_accuracy: 0.6556 - val_loss: 0.8897
Epoch 2/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 25s 73ms/step - accuracy: 0.6881 - loss: 0.8104 - val_accuracy: 0.7998 - val_loss: 0.5936
Epoch 3/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 26s 75ms/step - accuracy: 0.8088 - loss: 0.5849 - val_accuracy: 0.8180 - val_loss: 0.5221
Epoch 4/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 25s 74ms/step - accuracy: 0.8421 - loss: 0.4869 - val_accuracy: 0.8455 - val_loss: 0.4828
Epoch 5/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 25s 74ms/step - accuracy: 0.8269 - loss: 0.5173 - val_accuracy: 0.8284 - val_loss: 0.5109
Epoch 6/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 26s 77ms/step - accuracy: 0.8621 - loss: 0.4353 - val_accuracy: 0.8467 - val_loss: 0.4812
Epoch 7/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 25s 75ms/step - accuracy: 0.8696 - loss: 0.4052 - val_accuracy: 0.8546 - val_loss: 0.4614
Epoch 8/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 26s 77ms/step - accuracy: 0.8735 - loss: 0.3975 -

In [145]:
loss, acc = model_lstm.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.3f}")

189/189 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.8651 - loss: 0.4260
Test Accuracy: 0.865


# ***RNN***

In [146]:
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout

model_rnn = Sequential()
model_rnn.add(SimpleRNN(128, input_shape=(X_train.shape[1], X_train.shape[2])))
model_rnn.add(Dropout(0.3))
model_rnn.add(Dense(y_train.shape[1], activation='softmax'))

model_rnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_rnn.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 128)            │        54,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 55,557 (217.02 KB)

 Trainable params: 55,557 (217.02 KB)

 Non-trainable params: 0 (0.00 B)

In [147]:
history_rnn = model_rnn.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=10,
    batch_size=64
)

Epoch 1/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.5827 - loss: 1.1151 - val_accuracy: 0.5625 - val_loss: 1.1950
Epoch 2/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 8s 23ms/step - accuracy: 0.5880 - loss: 1.1983 - val_accuracy: 0.4720 - val_loss: 1.3363
Epoch 3/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 8s 23ms/step - accuracy: 0.5135 - loss: 1.2698 - val_accuracy: 0.3822 - val_loss: 1.4291
Epoch 4/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 10s 28ms/step - accuracy: 0.4657 - loss: 1.3286 - val_accuracy: 0.5098 - val_loss: 1.3113
Epoch 5/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 8s 23ms/step - accuracy: 0.3311 - loss: 1.5570 - val_accuracy: 0.3930 - val_loss: 1.4510
Epoch 6/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step - accuracy: 0.3850 - loss: 1.4662 - val_accuracy: 0.4474 - val_loss: 1.3459
Epoch 7/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step - accuracy: 0.3946 - loss: 1.4308 - val_accuracy: 0.3037 - val_loss: 1.5504
Epoch 8/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 8s 23ms/step - accuracy: 0.3527 - loss: 1.4881 - val_ac

In [148]:
loss_rnn, acc_rnn = model_rnn.evaluate(X_test, y_test)
print(f"Test Accuracy (RNN): {acc_rnn:.3f}")

189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.3541 - loss: 1.4952
Test Accuracy (RNN): 0.354


# ***BERT***

In [149]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf

model_name = "aubmindlab/bert-base-arabertv2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [150]:
from sklearn.model_selection import train_test_split
le.fit(df['name_ar'])
subset_df = df.sample(n=5000, random_state=42)

texts = subset_df['question_body'].tolist()  # Use raw text data
labels = le.fit_transform(subset_df['name_ar'])

X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, stratify=labels, random_state=42)

In [151]:
train_encodings = tokenizer(X_train, truncation=True, padding=True, return_tensors='tf')
test_encodings = tokenizer(X_test, truncation=True, padding=True, return_tensors='tf')

In [152]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
)).batch(16)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
)).batch(16)


In [153]:
from transformers import create_optimizer
# optimizer and learning rate scheduler
batch_size = 16
epochs = 5
steps_per_epoch = len(train_dataset)
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1 * num_train_steps)

optimizer, schedule = create_optimizer(
    init_lr=5e-5,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps
)

model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [154]:
model.fit(train_dataset, validation_data=test_dataset, epochs=3)

Epoch 1/3
250/250 [==============================] - 732s 3s/step - loss: 0.9644 - accuracy: 0.6260 - val_loss: 0.4397 - val_accuracy: 0.8450
Epoch 2/3
250/250 [==============================] - 660s 3s/step - loss: 0.3869 - accuracy: 0.8712 - val_loss: 0.3730 - val_accuracy: 0.8810
Epoch 3/3
250/250 [==============================] - 635s 3s/step - loss: 0.2625 - accuracy: 0.9162 - val_loss: 0.3513 - val_accuracy: 0.8840


In [156]:
import numpy as np
from sklearn.metrics import classification_report

pred_logits = model.predict(test_dataset).logits
y_pred = np.argmax(pred_logits, axis=1)

report = classification_report(y_test, y_pred, target_names=le.classes_, output_dict=True)
pd.DataFrame(report).transpose().round(3)

63/63 [==============================] - 43s 679ms/step


,precision,recall,f1-score,support
الطب النفسي,0.794,0.830,0.812,200.000
تغذية,0.877,0.871,0.874,155.000
جراحة العظام والمفاصل,0.914,0.901,0.907,272.000
طب اسنان,0.922,0.945,0.934,201.000
طب عيون,0.908,0.860,0.884,172.000
accuracy,0.884,0.884,0.884,0.884
macro avg,0.883,0.881,0.882,1000.000
weighted avg,0.885,0.884,0.884,1000.000


# ***GPT***

In [157]:
from transformers import TFGPT2ForSequenceClassification, AutoTokenizer, create_optimizer
model_name = "aubmindlab/aragpt2-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # GPT2 doesn't have a pad token, so we use eos_token

In [158]:
model = TFGPT2ForSequenceClassification.from_pretrained(
    model_name,
    num_labels=5,
    ignore_mismatched_sizes=True
)
model.config.pad_token_id = tokenizer.pad_token_id

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2ForSequenceClassification: ['h.2.attn.masked_bias', 'h.1.attn.masked_bias', 'h.7.attn.masked_bias', 'h.6.attn.masked_bias', 'h.5.attn.masked_bias', 'h.0.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.3.attn.masked_bias', 'h.11.attn.masked_bias', 'h.4.attn.masked_bias', 'h.8.attn.masked_bias']
- This IS expected if you are initializing TFGPT2ForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2ForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFGPT2ForSequenceClassification were not initialized from the P

In [159]:
train_encodings = tokenizer(X_train, truncation=True, padding=True, return_tensors='tf')
test_encodings = tokenizer(X_test, truncation=True, padding=True, return_tensors='tf')


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [160]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    tf.convert_to_tensor(y_train)
)).batch(16)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    tf.convert_to_tensor(y_test)
)).batch(16)


In [161]:
# Optimizer
batch_size = 16
epochs = 3
steps_per_epoch = len(train_dataset)
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1 * num_train_steps)

optimizer, lr_schedule = create_optimizer(
    init_lr=5e-5,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
)

In [162]:
# Compile model
model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [163]:
history_GPT = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=epochs
)

Epoch 1/3
250/250 [==============================] - 692s 3s/step - loss: 1.3168 - accuracy: 0.4620 - val_loss: 0.7036 - val_accuracy: 0.7600
Epoch 2/3
250/250 [==============================] - 673s 3s/step - loss: 0.6645 - accuracy: 0.7705 - val_loss: 0.5245 - val_accuracy: 0.8250
Epoch 3/3
250/250 [==============================] - 665s 3s/step - loss: 0.4976 - accuracy: 0.8280 - val_loss: 0.4623 - val_accuracy: 0.8540


In [165]:
pred_logits = model.predict(test_dataset).logits
y_pred = np.argmax(pred_logits, axis=1)

report = classification_report(y_test, y_pred, target_names=le.classes_,output_dict=True)
pd.DataFrame(report).transpose().round(3)

63/63 [==============================] - 46s 732ms/step


,precision,recall,f1-score,support
الطب النفسي,0.762,0.800,0.780,200.000
تغذية,0.830,0.852,0.841,155.000
جراحة العظام والمفاصل,0.903,0.886,0.894,272.000
طب اسنان,0.885,0.915,0.900,201.000
طب عيون,0.878,0.797,0.835,172.000
accuracy,0.854,0.854,0.854,0.854
macro avg,0.852,0.850,0.850,1000.000
weighted avg,0.855,0.854,0.854,1000.000
